In [1]:
# 3. CNN으로 MNIST 분류하기

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU사용 가능할 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

cuda


In [3]:
# 학습에 사용될 파라미터

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# 데이터로더를 사용하여 데이터를 다루기 위해서 데이터셋 정의

mnist_train = dsets.MNIST(root='C:/Users/gih54/Documents/MNIST_data/', # 데이터 다운로드 경로
                          train=True, # train 인자 true : 훈련 데이터, false : 테스트 데이터
                          transform=transforms.ToTensor(), # transform 은 현재 데이터를 파이토치 텐서로 변환
                          download=False)

mnist_test = dsets.MNIST(root='C:/Users/gih54/Documents/MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=False)

In [5]:
# 데이터로더를 사용해서 배치 크기 지정

data_loader = torch.utils.data.DataLoader(
    dataset= mnist_train,
    batch_size= batch_size,
    shuffle= True,
    drop_last= True)

In [6]:
# 클래스로 모델을 설계

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape = (?, 28, 28, 1)
        # Conv -> (?, 28, 28, 32)
        # Pool -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size= 3, stride= 1, padding= 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size= 2, stride= 2)
        )

        # 두번째층
        # ImgIn shape = (?, 28, 28, 1)
        # Conv -> (?, 28, 28, 64)
        # Pool -> (?, 14, 14, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size= 3, stride= 1, padding= 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size= 2, stride= 2)
        )

        # 전결합층 7 X 7 X 64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias= True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [7]:
# 모델 정의
# CNN 모델 정의

model = CNN().to(device)

In [8]:
# 비용함수 및 옵티마이저 정의

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [9]:
# 총 배치의 수 출력

total_batch = len(data_loader)
print(f"총 배치의 수 : {total_batch}")

총 배치의 수 : 600


In [10]:
# 모델 훈련
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니배치, Y는 레이블
        # image is already size of (28 X 28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print("[Epoch: {:>4}] cost = {:>9}".format(epoch+1, avg_cost))

[Epoch:    1] cost = 0.22568120062351227
[Epoch:    2] cost = 0.06312915682792664
[Epoch:    3] cost = 0.04635269194841385
[Epoch:    4] cost = 0.03754287585616112
[Epoch:    5] cost = 0.031568095088005066
[Epoch:    6] cost = 0.026179123669862747
[Epoch:    7] cost = 0.02203834243118763
[Epoch:    8] cost = 0.0186180230230093
[Epoch:    9] cost = 0.01626327447593212
[Epoch:   10] cost = 0.013573218137025833
[Epoch:   11] cost = 0.00987319741398096
[Epoch:   12] cost = 0.010237324051558971
[Epoch:   13] cost = 0.008427703753113747
[Epoch:   14] cost = 0.00642367685213685
[Epoch:   15] cost = 0.007604363840073347


In [13]:
# 테스트
# 학습을 진행하지 않을 것이므로 torch.no_grad()

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())

Accuracy:  0.9872999787330627
